# Loading and Exploring the DataFrames


In [2]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv("/home/abdallah/Courses/Kiwilytics_DE/Kiwilytics_DE_Docker/python/Orders_with_issues.csv")
# print(df.head())
# df.info()
# UNIQUE VALUES
df["ShippingCompany"].value_counts()


ShippingCompany
FedEx Logistics                   54
DHL Express                       53
Aramex International              51
UPS Worldwide                     46
Kiwilytics Goods Shipping LLC.    41
Name: count, dtype: int64

# cleaning Dates, numeric entries

In [41]:
# convert Dates
dates = df["OrderDate"].copy()
dates = pd.to_datetime(df["OrderDate"], format="%Y-%m-%d", errors="coerce")
df["OrderDate"] = dates

In [72]:
costs = df["ShippingCost"].copy()
costs = pd.to_numeric(df["ShippingCost"], errors="coerce")
# replace if costs smaller than 0 then is 0
df.loc[costs < 0, "ShippingCost"] = np.nan #or costs[costs < 0] = 0 o
df["ShippingCost"] = costs
# df["ShippingCost"] = costs


# Handling Nulls, Fixed Fields & Standardization


In [86]:
df["OrderID"] = df["OrderID"].fillna(method="ffill")
df["OrderID"].isna().sum()
df["CustomerID"].fillna("Unknown", inplace=True)

/tmp/ipykernel_28267/2934179393.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["OrderID"] = df["OrderID"].fillna(method="ffill")


0

In [92]:
# Standardization
df["ShippingCompany"].str.strip().str.title()
df["ShipCountry"].str.strip().str.title()
df["ShipCity"].str.strip()

ShippingCompany
FedEx Logistics                   54
DHL Express                       53
Aramex International              51
UPS Worldwide                     46
Kiwilytics Goods Shipping LLC.    41
Name: count, dtype: int64

In [95]:
company = df["ShippingCompany"]
df.loc[company.str.contains("UPS", case=False, regex=True), "ShippingCompany"] = "UPS"
company.value_counts()

ShippingCompany
FedEx Logistics                   54
DHL Express                       53
Aramex International              51
UPS                               46
Kiwilytics Goods Shipping LLC.    41
Name: count, dtype: int64

# Feature Engineering

In [115]:
# n of orders that are delayed of 15 days
df["ShippedDate"] = pd.to_datetime(df["ShippedDate"], format="%Y-%m-%d", errors="coerce")
df["OrderDate"] = pd.to_datetime(df["OrderDate"], format="%Y-%m-%d", errors="coerce")
df["IsDelayed"] = (df["ShippedDate"] - df["OrderDate"] > pd.Timedelta(days=15))
df["IsDelayed"]
# df[df["IsDelayed"] > 15].shape[0]


0       True
1       True
2       True
3       True
4       True
       ...  
240     True
241    False
242     True
243     True
244     True
Name: IsDelayed, Length: 245, dtype: bool

# Grouping

In [122]:
avg_shipping = df.groupby("ShippingCompany")["ShippingCost"].mean()
avg_shipping

ShippingCompany
Aramex International              258.805111
DHL Express                       233.160000
FedEx Logistics                   229.533269
Kiwilytics Goods Shipping LLC.    245.112162
UPS                               258.632791
Name: ShippingCost, dtype: float64

In [131]:
df.head()

,OrderID,CustomerID,OrderDate,ShippedDate,ShippingCost,ShipCountry,ShipCity,ShippingCompany,IsDelayed
0,1000.0,C001,2025-05-17,2025-07-30,NaN,Germany,Hamburg,Kiwilytics Goods Shipping LLC.,True
1,1001.0,C002,2025-01-26,2025-07-30,320.61,Canada,Montreal,UPS,True
2,1002.0,C003,2025-03-08,2025-07-30,165.17,Canada,Vancouver,FedEx Logistics,True
3,1003.0,C004,2025-03-24,2025-07-30,12.55,Germany,Munich,Aramex International,True
4,1004.0,C005,2025-04-15,2025-07-30,186.36,Canada,Vancouver,FedEx Logistics,True


# Export, Report And Wrapping

In [135]:
df.to_csv("/home/abdallah/Courses/Kiwilytics_DE/Kiwilytics_DE_Docker/python/cleaned_orders_final.csv", index=False)
print(df.head())

df["ShippingCompany"].value_counts().head(3)

   OrderID CustomerID  OrderDate ShippedDate  ShippingCost ShipCountry  \
0   1000.0       C001 2025-05-17  2025-07-30           NaN     Germany   
1   1001.0       C002 2025-01-26  2025-07-30        320.61      Canada   
2   1002.0       C003 2025-03-08  2025-07-30        165.17      Canada   
3   1003.0       C004 2025-03-24  2025-07-30         12.55     Germany   
4   1004.0       C005 2025-04-15  2025-07-30        186.36      Canada   

    ShipCity                 ShippingCompany  IsDelayed  
0    Hamburg  Kiwilytics Goods Shipping LLC.       True  
1   Montreal                             UPS       True  
2  Vancouver                 FedEx Logistics       True  
3     Munich            Aramex International       True  
4  Vancouver                 FedEx Logistics       True  


ShippingCompany
FedEx Logistics         54
DHL Express             53
Aramex International    51
Name: count, dtype: int64